# Web scraping for RAJA

## Import des packages

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
from selenium.common.exceptions import NoSuchElementException

## Input

In [2]:
"https://www.manutan.fr/fr/maf/brand/{marque_field}?krypto=57%2FPsCvM9kiDHrvSTHgjP1xkSlCM3v2cfg1A0jeneRIj04HtS7IPawyXdCQPJWEwGbBQMsHFfeij6A5MBEXFYfia6ea4iAU%2B2w07x6DUCC3mevyGpti5HKkXxN%2BgLTOg#productBeginIndex:{max_product}&orderBy:0&"

'https://www.manutan.fr/fr/maf/brand/{marque_field}?krypto=57%2FPsCvM9kiDHrvSTHgjP1xkSlCM3v2cfg1A0jeneRIj04HtS7IPawyXdCQPJWEwGbBQMsHFfeij6A5MBEXFYfia6ea4iAU%2B2w07x6DUCC3mevyGpti5HKkXxN%2BgLTOg#productBeginIndex:{max_product}&orderBy:0&'

In [32]:
input_df = pd.DataFrame({"website":["bernard","jpg","raja","manutan","bruneau"],
              "url":["https://www.bernard.fr/?query={marque_field}&refinementList%5BMarque%20gamme%20web%5D%5B0%5D={marque_field}&SearchCat=natural&SearchQuery={marque_field}",
                     "https://www.jpg.fr/?query={marque_field}&refinementList%5BMarque%20gamme%20web%5D%5B0%5D={marque_field}&SearchCat=natural&SearchQuery={marque_field}",
                     "https://www.raja.fr/?query={marque_field}&refinementList%5BZ_CARACT_ATT_014%5D%5B0%5D={marque_field}&SearchCat=natural&SearchQuery={marque_field}",
                     "https://www.manutan.fr/SearchDisplay?categoryId=&storeId=11701&catalogId=11604&langId=-2&sType=SimpleSearch&resultCatEntryType=2&showResultsPage=true&searchSource=Q&pageView=&beginIndex=0&pageSize=28&isSearchDFType=ES&searchTerm={marque_field}&authToken=109810278%252CzwSxxg0us%252B7%252BT7tKqbwrdK2IL1SvC2bD8BElZz45B%252Fc%253D#productBeginIndex:{max_product}&orderBy:0&",
                     "https://www.bruneau.fr/search?page={num_page}&term={marque_field}"],
               "xpath_cookies_accept":["/html/body/div[4]/div[2]/div[1]/button[2]",
                         "/html/body/div[4]/div[2]/div[1]/button[2]",
                         "/html/body/div[4]/div[2]/div[1]/button[2]",
                         "/html/body/div[6]/div[3]/div[1]/div[1]/div[2]/div[1]/div[1]/button[2]",
                         "/html/body/div[3]/div[2]/div[1]/div[2]/div[1]/div[1]/div[2]/div[1]/div[1]/button[1]"],
               "xpath_load_more_button":["/html/body/div[5]/div[1]/div[1]/div[3]/div[4]/button[1]",
                                   "/html/body/div[5]/div[1]/div[1]/div[3]/div[4]/button[1]",
                                   "/html/body/div[5]/div[1]/div[1]/div[3]/div[4]/button[1]",
                                   None,
                                   "/html/body/div[1]/div[2]/div[1]/div[1]/div[2]/div[2]/div[2]/a[1]"],
                  
               "class_price":[("span","bold Price_overline Price_overline-yellow"),
                              ("span","bold Price_overline Price_overline-yellow"),
                              ("span","bold Price_overline Price_overline-yellow"),
                              ("span","price"),
                              ("span","isg-gros-text-arial-gras")],
               "class_name":[("span","Product-title text-black"),
                             ("span","Product-title text-black"),
                             ("span","Product-title text-black"),
                             ("a","prodlist-item"),
                             ("p","isg-catalog-product-title")],
               "class_ref":[("span","Product-sku"),
                            ("span","Product-sku"),
                            ("span","Product-sku"),
                            ("div","product_name"),
                            ("div","isg-catalog-product")],
               "other":[None,None,None,None,"/html/body/div[1]/div[3]/div[2]/div[1]/button[1]"]
                     })
input_df

website                                                url  \
0  bernard  https://www.bernard.fr/?query={marque_field}&r...   
1      jpg  https://www.jpg.fr/?query={marque_field}&refin...   
2     raja  https://www.raja.fr/?query={marque_field}&refi...   
3  manutan  https://www.manutan.fr/SearchDisplay?categoryI...   
4  bruneau  https://www.bruneau.fr/search?page={num_page}&...   

                                xpath_cookies_accept  \
0          /html/body/div[4]/div[2]/div[1]/button[2]   
1          /html/body/div[4]/div[2]/div[1]/button[2]   
2          /html/body/div[4]/div[2]/div[1]/button[2]   
3  /html/body/div[6]/div[3]/div[1]/div[1]/div[2]/...   
4  /html/body/div[3]/div[2]/div[1]/div[2]/div[1]/...   

                              xpath_load_more_button  \
0  /html/body/div[5]/div[1]/div[1]/div[3]/div[4]/...   
1  /html/body/div[5]/div[1]/div[1]/div[3]/div[4]/...   
2  /html/body/div[5]/div[1]/div[1]/div[3]/div[4]/...   
3                                               None   
4  /html/body/div[1]/div[2]/div[1]/div[1]/div[2]/...   

                                         class_price  \
0  (span, bold Price_overline Price_overline-yellow)   
1  (span, bold Price_overline Price_overline-yellow)   
2  (span, bold Price_overline Price_overline-yellow)   
3                                      (span, price)   
4                   (span, isg-gros-text-arial-gras)   

                         class_name                   class_ref  \
0  (span, Product-title text-black)         (span, Product-sku)   
1  (span, Product-title text-black)         (span, Product-sku)   
2  (span, Product-title text-black)         (span, Product-sku)   
3                (a, prodlist-item)         (div, product_name)   
4    (p, isg-catalog-product-title)  (div, isg-catalog-product)   

                                              other  
0                                              None  
1                                              None  
2                                              None  
3                                              None  
4  /html/body/div[1]/div[3]/div[2]/div[1]/button[1]

## Test unitaire

In [33]:
s = Service('chromedriver.exe')
chromeOptions = Options()
# chromeOptions.add_argument("--headless")
# chromeOptions.add_argument("--blink-settings=imagesEnabled=false")

# Adding argument to disable the AutomationControlled flag 
chromeOptions.add_argument("--disable-blink-features=AutomationControlled") 
 
# Exclude the collection of enable-automation switches 
chromeOptions.add_experimental_option("excludeSwitches", ["enable-automation"]) 


# Turn-off userAutomationExtension 
chromeOptions.add_experimental_option("useAutomationExtension", False) 

driver = webdriver.Chrome(service=s, options=chromeOptions)
driver.implicitly_wait(10)
marques = ["TORK"] #["TORK","JEX","ST MARC","HARPIC","AJAX","ROSSIGNOL","ANSELL","BLAKLADER"]
soup_df = pd.DataFrame()


websites = ["bruneau"]

for website in websites:
    print(f"----------------------Connecting to {website}-------------------------")
    for marque in marques :
        marque = marque.replace(" ","%20")
        if website == "manutan":
            marque = marque.replace(" ","+")
        query = input_df.loc[input_df["website"]==websites[0],"url"].values[0].format(marque_field = marque, num_page=1)
        driver.get(query)

        #Cookies
        try:
            driver.find_element(By.XPATH,input_df.loc[input_df["website"]==websites[0],"xpath_cookies_accept"].values[0]).click()
        except NoSuchElementException:
            pass

        #Promo
        try:
            driver.find_element(By.XPATH,input_df.loc[input_df["website"]==websites[0],"other"].values[0]).click()
        except NoSuchElementException:
            pass
        
        if website == "manutan":
            html_by_page = []

            soup = BeautifulSoup(driver.page_source, 'html.parser')
            html_by_page.append(soup)
            
            match_html = soup.find("span",class_="num_products")
            if match_html:
                match = re.search(r'sur\s+(\d+)', match_html.text)
                nb_products = int(match.group(1))
                nb_pages = int(nb_products / 28) + 1
                for nb_page in range(1,nb_pages):
                    query = input_df.loc[input_df["website"]==websites[0],"url"].values[0].format(marque_field = marque, max_product=nb_page * 28)    
                    driver.get(query)
                    try:
                        driver.find_element(By.XPATH,input_df.loc[input_df["website"]==websites[0],"xpath_cookies_accept"].values[0]).click()
                    except NoSuchElementException:
                        pass
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    html_by_page.append(soup)

            marque = marque.replace("%20"," ")
            soup_df = pd.concat([soup_df, pd.DataFrame({"website": website,
                                                     "marque": marque,
                                                     "html_content": html_by_page},index=list(range(len(html_by_page))))]).reset_index(drop=True)

        else:
            html_by_page = []
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            html_by_page.append(soup)
            match_html = soup.find("h1",class_="isg-titre-camingo")
            match = match_html.text
            nb_results = int(re.search(r'(\d+)\s+', match).group(0))
            n_resuls_by_page = 48
            if nb_results > n_resuls_by_page :
                nb_pages = int(nb_results / n_resuls_by_page) + 1
                for nb_page in range(2,nb_pages + 1):
                    query = input_df.loc[input_df["website"]==website,"url"].values[0].format(marque_field = marque, num_page=nb_page)    
                    driver.get(query)
                    try:
                        driver.find_element(By.XPATH,input_df.loc[input_df["website"]==websites[0],"xpath_cookies_accept"].values[0]).click()
                    except NoSuchElementException:
                        pass
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    html_by_page.append(soup)

            soup_df = pd.concat([soup_df, pd.DataFrame({"website": website,
                                                     "marque": marque,
                                                     "html_content": html_by_page},index=list(range(len(html_by_page))))]).reset_index(drop=True)
            
        print(f" --- {marque} products collected")
print("\n","Data collection complete !")
print(soup_df)

----------------------Connecting to bruneau-------------------------
 --- TORK products collected

 Data collection complete !
   website marque                                       html_content
0  bruneau   TORK  [[[\n, <link as="font" crossorigin="" href="/d...
1  bruneau   TORK  [[[\n, <link as="font" crossorigin="" href="/d...
2  bruneau   TORK  [[[\n, <link as="font" crossorigin="" href="/d...


In [ ]:
"                102 résultats pour « TORK »"

## Code refactoring

In [38]:
def html_collection(website, marque,soup_df):
    
    #Dealing with spaces in marque
    marque = marque.replace(" ","%20")

    #Manutan works with paging, while the other work with loadmore button, so we need to precise that we want the page 0
    if website == "manutan":
        marque = marque.replace(" ","+")
        query=input_df.loc[input_df["website"]==website,"url"].values[0].format(marque_field = marque, max_product=0)
    elif website == "bruneau":
        query=input_df.loc[input_df["website"]==website,"url"].values[0].format(marque_field = marque, num_page=1)
    else:
        query=input_df.loc[input_df["website"]==website,"url"].values[0].format(marque_field = marque)

    #Opening web page
    driver.get(query)

    #Clicking on the accept cookies button if any
    try:
        driver.find_element(By.XPATH,input_df.loc[input_df["website"]==website,"xpath_cookies_accept"].values[0]).click()
    except NoSuchElementException:
        pass
    
    #Closing the promotion window for bruneau, if any
    if website == "bruneau":
        try:
            driver.find_element(By.XPATH,input_df.loc[input_df["website"]==websites[0],"other"].values[0]).click()
        except:
            pass
    
    #Collecting the html code
    if website == "manutan":
        
        html_by_page = []
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        html_by_page.append(soup)
        match_html = re.search(r'sur\s+(\d+)', soup.find("span",class_="num_products").text)
        if match_html:
            # match = re.search(r'sur\s+(\d+)', match_html.text)
            nb_products = int(match_html.group(1))
            nb_pages = int(nb_products / 28) + 1
            for nb_page in range(1,nb_pages):
                query = input_df.loc[input_df["website"]==website,"url"].values[0].format(marque_field = marque, max_product=nb_page * 28)    
                driver.get(query)
                try:
                    driver.find_element(By.XPATH,input_df.loc[input_df["website"]==website,"xpath_cookies_accept"].values[0]).click()
                except:
                    pass
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                html_by_page.append(soup)

        marque = marque.replace("%20"," ")
        soup_df = pd.concat([soup_df, pd.DataFrame({"website": website,
                                                     "marque": marque,
                                                     "html_content": html_by_page},index=list(range(len(html_by_page))))]).reset_index(drop=True)


    elif website == "bruneau":
        
        html_by_page = []
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        html_by_page.append(soup)
        match_html = soup.find("h1",class_="isg-titre-camingo")
        match = match_html.text
        nb_results = int(re.search(r'(\d+)\s+', match).group(0))
        n_resuls_by_page = 48
        if nb_results > n_resuls_by_page :
            nb_pages = int(nb_results / n_resuls_by_page) + 1
            for nb_page in range(2,nb_pages + 1):
                query = input_df.loc[input_df["website"]==website,"url"].values[0].format(marque_field = marque, num_page=nb_page)    
                driver.get(query)
                try:
                    driver.find_element(By.XPATH,input_df.loc[input_df["website"]==website,"xpath_cookies_accept"].values[0]).click()
                except NoSuchElementException:
                    pass
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                html_by_page.append(soup)

        soup_df = pd.concat([soup_df, pd.DataFrame({"website": website,
                                                    "marque": marque,
                                                    "html_content": html_by_page},index=list(range(len(html_by_page))))]).reset_index(drop=True)

    else:
        e = True
        while e == True:
            try:
                loadMoreButton = driver.find_element(By.XPATH,input_df.loc[input_df["website"]==website,"xpath_load_more_button"].values[0])
                time.sleep(8)
                loadMoreButton.click()
                time.sleep(8)
                e = True
            except Exception:
                e = False
    
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        marque = marque.replace("%20"," ")
        soup_df = pd.concat([soup_df, pd.DataFrame({"website": website,
                                                        "marque": marque,
                                                        "html_content": soup},index=[0])]).reset_index(drop=True)
            
    return soup_df


In [40]:
s = Service('chromedriver.exe')
chromeOptions = Options()

chromeOptions.add_argument("--headless")
chromeOptions.add_argument("--blink-settings=imagesEnabled=false")


# Adding argument to disable the AutomationControlled flag 
chromeOptions.add_argument("--disable-blink-features=AutomationControlled") 
 
# Exclude the collection of enable-automation switches 
chromeOptions.add_experimental_option("excludeSwitches", ["enable-automation"]) 
 
# Turn-off userAutomationExtension 
chromeOptions.add_experimental_option("useAutomationExtension", False) 

driver = webdriver.Chrome(service=s, options=chromeOptions)
driver.implicitly_wait(10)
marques = ["TORK"] #["TORK","JEX","ST MARC","HARPIC","AJAX","ROSSIGNOL","ANSELL","BLAKLADER"]

soup_df = pd.DataFrame()
websites = ["bruneau"] #["bruneau","jpg","raja","manutan","bernard"]

for website in websites:
    print(f"----------------------Connecting to {website}-------------------------")
    for marque in marques :
        soup_df = html_collection(website, marque, soup_df)
        print(f" --- {marque} products collected")
print("\n","Data collection complete !")

----------------------Connecting to bruneau-------------------------
 --- TORK products collected

 Data collection complete !


In [7]:
def clean_prices(price: str):
    price = price.replace(",",".").replace("€","").replace(" ht","").rstrip().replace(" ","").replace("\xa0","").replace("\u202f","")
    return price

In [8]:
def prices_collection(soup_df):

    results_df = pd.DataFrame()

    for index, row in soup_df.iterrows():
            
            website = row["website"]
            marque = row["marque"]
            html_content = row["html_content"]

            print(f"------------------extracting prices of {marque} from {website}---------------------")

            #Collecting products names
            results_names = html_content.find_all(input_df.loc[input_df["website"]==website,"class_name"].values[0][0],
                                                class_=input_df.loc[input_df["website"]==website,"class_name"].values[0][1])

            if website == "manutan":
                product_names = []
                for item in results_names:
                    try:
                        product_names.append(item["aria-label"])
                    except KeyError:
                        pass
            else:
                product_names = [product.text for product in results_names]

            #Collecting products references
            results_ref = html_content.find_all(input_df.loc[input_df["website"]==website,"class_ref"].values[0][0],
                                                class_=input_df.loc[input_df["website"]==website,"class_ref"].values[0][1])
            
            if website == "manutan":
                product_ref = [item["id"].lstrip("product_name_") for item in results_ref]
            elif website == "bruneau":
                product_ref = [product["id"].strip("product-") for product in results_ref]
            else:
                product_ref = [product.text for product in results_ref]

            #Collecting products prices
            results_prices = html_content.find_all(input_df.loc[input_df["website"]==website,"class_price"].values[0][0],
                                                class_=input_df.loc[input_df["website"]==website,"class_price"].values[0][1])
            
            if website == "bruneau":
                results_prices = [results_prices[i] for i in range(0,len(results_prices),2)]

            product_prices = [price.text for price in results_prices]

            #Collecting and storing data into a dataframe, if data found
            if (len(product_names) == 0) | (len(product_ref) == 0) | (len(product_prices) == 0):
                 print(f"------------------No results found---------------------")
            else:
                results_marque_df = pd.DataFrame({"Site": website,
                                    "Marque": marque,
                                    "Produit": product_names,
                                    "Référence": product_ref,
                                    "Prix": product_prices})
                results_df = pd.concat([results_df,results_marque_df])
                results_df.reset_index(drop=True,inplace=True)

                print(f"------------------extraction finished---------------------")

    #Price formating
    # results_df["Prix"] = results_df["Prix"].apply(lambda x : clean_prices(x))
    # results_df["Prix"] = round(pd.to_numeric(results_df["Prix"]),2)

    return results_df


In [41]:
test = prices_collection(soup_df)
test["Prix_clean"] = test["Prix"].apply(clean_prices)
test["Prix_clean_clean"] = round(pd.to_numeric(test["Prix_clean"]),2)


------------------extracting prices of TORK from bruneau---------------------
------------------extraction finished---------------------
------------------extracting prices of TORK from bruneau---------------------
------------------extraction finished---------------------
------------------extracting prices of TORK from bruneau---------------------
------------------extraction finished---------------------


In [42]:
test.to_excel("test_final_bruneau.xlsx",index=False)

In [32]:
test.loc[985,"Prix_clean"]

'1\u202f209.00'